# Load Data

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForTokenClassification, BertTokenizerFast
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
import numpy as np
from tqdm import tqdm

In [1]:
data = pd.read_csv('../ner.csv')
data.head()

,description,cooking_steps,ingredients,full
0,украсить обычный бутерброд красный икра,сделать бутерброд красный икра тонко порезать ...,"батон, икра красный, маслина, лимон, зелень, м...",украсить обычный бутерброд красный икра сделат...
1,обжарить баклажан жареный гриб сметанный соус,баклажан промыть нарезать крупный кусочек посо...,"баклажан, гриб, лук репчатый, сметана, мука пш...",обжарить баклажан жареный гриб сметанный соус ...
2,отбить кусочек мясо обжарить панировочный сухарь,яйцо несильно взбить подготовить панировочный ...,"свинина, сухарь панировочный, яйцо куриный, со...",отбить кусочек мясо обжарить панировочный суха...
3,вообще это банкетный блюдо думать подать вечер...,свежий щука почистить повредить кожа вырезать ...,"щука, батон, яйцо куриный, масло сливочный, мо...",вообще это банкетный блюдо думать подать вечер...
4,фарш вкус виноградный лист,виноградный лист промыть ошпарить кипяток лук ...,"фарш мясной, лук репчатый, морковь, лист виног...",фарш вкус виноградный лист виноградный лист пр...


Получим из строки, содержащей ингредиенты через запятую, список:

In [4]:
def split_ingredients(ingredients: str):
    return str(ingredients).split(', ')

data['ingredients'] = data['ingredients'].apply(split_ingredients)

In [5]:
data.head()

,description,cooking_steps,ingredients,full
0,украсить обычный бутерброд красный икра,сделать бутерброд красный икра тонко порезать ...,"[батон, икра красный, маслина, лимон, зелень, ...",украсить обычный бутерброд красный икра сделат...
1,обжарить баклажан жареный гриб сметанный соус,баклажан промыть нарезать крупный кусочек посо...,"[баклажан, гриб, лук репчатый, сметана, мука п...",обжарить баклажан жареный гриб сметанный соус ...
2,отбить кусочек мясо обжарить панировочный сухарь,яйцо несильно взбить подготовить панировочный ...,"[свинина, сухарь панировочный, яйцо куриный, с...",отбить кусочек мясо обжарить панировочный суха...
3,вообще это банкетный блюдо думать подать вечер...,свежий щука почистить повредить кожа вырезать ...,"[щука, батон, яйцо куриный, масло сливочный, м...",вообще это банкетный блюдо думать подать вечер...
4,фарш вкус виноградный лист,виноградный лист промыть ошпарить кипяток лук ...,"[фарш мясной, лук репчатый, морковь, лист вино...",фарш вкус виноградный лист виноградный лист пр...


# Разметка данных

In [16]:
def create_bio_markup(text, ingredients):
    """
    Создает BIO-разметку для текста на основе списка ингредиентов
    
    Args:
        text (str): Исходный текст
        ingredients (list): Список ингредиентов для поиска
    Return:
        list: Список BIO-тегов для каждого слова в тексте
    """
    words = text.split()
    
    bio_tags = ['O'] * len(words)
    for ingredient in ingredients:
        ing_words = ingredient.split()
        
        # В прямом порядке -- "красный икра"
        for i in range(len(words) - len(ing_words) + 1):
            if ' '.join(words[i:i+len(ing_words)]).lower() == ingredient.lower():
                bio_tags[i] = 'B-ING'
                for j in range(1, len(ing_words)):
                    bio_tags[i+j] = 'I-ING'
        
        # В обратном порядке -- "икра красный"
        reversed_ing = ' '.join(reversed(ing_words))
        for i in range(len(words) - len(ing_words) + 1):
            if ' '.join(words[i:i+len(ing_words)]).lower() == reversed_ing.lower():
                bio_tags[i] = 'B-ING'
                for j in range(1, len(ing_words)):
                    bio_tags[i+j] = 'I-ING'
                    
    return bio_tags

Получим разметку в BIO - формате:

In [26]:
data['bio_tags'] = data.apply(lambda row: create_bio_markup(row['full'], row['ingredients']), axis=1)
data.head()

,description,cooking_steps,ingredients,full,bio_tags
0,украсить обычный бутерброд красный икра,сделать бутерброд красный икра тонко порезать ...,"[батон, икра красный, маслина, лимон, зелень, ...",украсить обычный бутерброд красный икра сделат...,"[O, O, O, B-ING, I-ING, O, O, B-ING, I-ING, O,..."
1,обжарить баклажан жареный гриб сметанный соус,баклажан промыть нарезать крупный кусочек посо...,"[баклажан, гриб, лук репчатый, сметана, мука п...",обжарить баклажан жареный гриб сметанный соус ...,"[O, B-ING, O, B-ING, O, O, B-ING, O, O, O, O, ..."
2,отбить кусочек мясо обжарить панировочный сухарь,яйцо несильно взбить подготовить панировочный ...,"[свинина, сухарь панировочный, яйцо куриный, с...",отбить кусочек мясо обжарить панировочный суха...,"[O, O, O, O, B-ING, I-ING, O, O, O, O, B-ING, ..."
3,вообще это банкетный блюдо думать подать вечер...,свежий щука почистить повредить кожа вырезать ...,"[щука, батон, яйцо куриный, масло сливочный, м...",вообще это банкетный блюдо думать подать вечер...,"[O, O, O, O, O, O, O, O, O, O, B-ING, O, O, O,..."
4,фарш вкус виноградный лист,виноградный лист промыть ошпарить кипяток лук ...,"[фарш мясной, лук репчатый, морковь, лист вино...",фарш вкус виноградный лист виноградный лист пр...,"[O, O, B-ING, I-ING, B-ING, I-ING, O, O, O, O,..."


Разделим данные на обучающую и тестовую выборки.

In [64]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=42)
print(train.shape, test.shape)

(123322, 5) (30831, 5)


Примеры работы функции по извлечению тегов:

In [18]:
create_bio_markup(data.iloc[0].values[3], data.iloc[0].values[2])

['O',
 'O',
 'O',
 'B-ING',
 'I-ING',
 'O',
 'O',
 'B-ING',
 'I-ING',
 'O',
 'O',
 'B-ING',
 'O',
 'O',
 'O',
 'B-ING',
 'O',
 'O',
 'B-ING',
 'O',
 'B-ING',
 'O',
 'O',
 'O',
 'O',
 'O']

In [19]:
bio_markup = create_bio_markup('Вкусное красное яблоко и яйцо на столе', ['красное яблоко', 'яйцо'])
words = dataset['text'].split()
for word, tag in zip(words, bio_markup):
    print(f"{word}: {tag}")

Вкусное: O
яблоко: B-ING
красное: I-ING
и: O
яйцо: B-ING
на: O
столе: O


# Train

In [67]:
class IngredientDataset(Dataset):
        def __init__(self, texts, tags, tokenizer, max_len=128):
            self.texts = texts
            self.tags = tags
            self.tokenizer = tokenizer
            self.max_len = max_len
            
            self.tag2idx = {'O': 0, 'B-ING': 1, 'I-ING': 2}
            self.idx2tag = {v: k for k, v in self.tag2idx.items()}
            
        def __len__(self):
            return len(self.texts)
            
        def __getitem__(self, idx):
            text = self.texts[idx]
            tags = self.tags[idx]
            
            # Токенизируем текст
            encoding = self.tokenizer(
                text,
                max_length=self.max_len,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
            
            word_ids = encoding.word_ids()
            labels = torch.ones(self.max_len, dtype=torch.long) * -100  # для спец. токенов
            
            current_word_idx = None
            for i, word_idx in enumerate(word_ids):
                if word_idx is None:
                    continue
                if word_idx != current_word_idx:
                    if word_idx < len(tags):
                        labels[i] = self.tag2idx[tags[word_idx]]
                        current_word_idx = word_idx
                else:
                    if word_idx < len(tags):
                        labels[i] = self.tag2idx[tags[word_idx]]
            
            return {
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'labels': labels
            }

In [68]:
tokenizer = BertTokenizerFast.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertForTokenClassification.from_pretrained(
        'DeepPavlov/rubert-base-cased',
        num_labels=3 
    )

Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [69]:
def make_loader(df):
    texts = df['full'].tolist()  
    tags = df['bio_tags'].tolist() 
    dataset = IngredientDataset(texts, tags, tokenizer)
    loader = DataLoader(dataset, batch_size=32, shuffle=True)
    return loader

train_loader = make_loader(train)
train_loader

In [70]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
# изменение оптимизатора незначительно повлияло на результат
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3

In [72]:
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
    for batch in progress_bar:
        optimizer.zero_grad()
            
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
            
        outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        labels=labels)
            
        loss = outputs.loss
        total_loss += loss.item()
            
        loss.backward()
        optimizer.step()
            
    progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})

    avg_loss = total_loss/len(train_loader)
    print(f"\nEpoch {epoch+1} completed. Average Loss: {avg_loss:.4f}")

Epoch 1/3: 100%|██████████| 3854/3854 [43:52<00:00,  1.46it/s]



Epoch 1 completed. Average Loss: 0.0718


Epoch 2/3: 100%|██████████| 3854/3854 [43:53<00:00,  1.46it/s]



Epoch 2 completed. Average Loss: 0.0515


Epoch 3/3: 100%|██████████| 3854/3854 [43:53<00:00,  1.46it/s]



Epoch 3 completed. Average Loss: 0.0455


# Test

In [73]:
    def predict_ingredients(text):
        model.eval()
        with torch.no_grad():
            inputs = tokenizer(
                text,
                return_tensors='pt',
                truncation=True,
                max_length=128,
                padding='max_length'
            )
            
            outputs = model(
                input_ids=inputs['input_ids'].to(device),
                attention_mask=inputs['attention_mask'].to(device)
            )
            
            predictions = torch.argmax(outputs.logits, dim=2)
            predicted_tags = []
            
            word_ids = inputs.word_ids(0)
            if word_ids is None:
                return []
                
            current_word_idx = None
            
            for idx, word_idx in enumerate(word_ids):
                if word_idx is None:
                    continue
                if word_idx != current_word_idx:
                    predicted_tags.append(train_dataset.idx2tag[predictions[0][idx].item()])
                    current_word_idx = word_idx
                    
            return predicted_tags

In [74]:
test_text = "На столе лежит зеленое яблоко и вареное яйцо"
predicted_tags = predict_ingredients(test_text)
words = test_text.split()
for word, tag in zip(words, predicted_tags):
    print(f"{word}: {tag}")

На: O
столе: O
лежит: O
зеленое: O
яблоко: B-ING
и: O
вареное: O
яйцо: O


In [86]:
test.iloc[15].values[3]

'просто вкусно '

In [92]:
test.iloc[15].values[4]

['O', 'O']

In [90]:
test_text = test.iloc[90].values[3]
predicted_tags = predict_ingredients(test_text)
words = test_text.split()
for word, tag in zip(words, predicted_tags):
    print(f"{word}: {tag}")

суп: O
взять: O
пёстрый: O
фасоль: B-ING
бульон: B-ING
использовать: O
который: O
предварительно: O
отваривать: O
крылышко: O
тот: O
индейка: O
другой: O
блюдо: O
крылышко: O
это: O
ласково: O
индейкито: O
какой: O
крылышко: O
дельтаплан: O
поэтому: O
бульон: B-ING
получиться: O
наваристый: O
достаточно: O
жирный: O
необычный: O
картошка: O
суп: O
зато: O
сельдерей: O
репа: B-ING
несколько: O
час: O
начало: O
готовка: O
замочить: O
фасоль: B-ING
залить: O
вода: O
едва: O
покрывало: O
это: O
время: O
сварить: O
бульон: B-ING
крыло: O
разделить: O
крыло: O
фаланга: O
залить: O
холодный: O
вода: O
довести: O
кипение: O
убавить: O
огонь: O
снять: O
пена: O
посолить: O
бросить: O
букет: O
гарни: O
любой: O
коренья: O
чёрный: O
перец: O
горошек: O
варить: O
среднее: O
огонь: O
час: O
полторадва: O
затем: O
вынуть: O
крыло: O
пригодиться: O
другой: O


In [94]:
type(predicted_tags)

list

In [104]:
test_predictions = []

for i, rows in tqdm(test.iterrows(), total=len(test), desc="Making predictions"):
    predicted_tags = predict_ingredients(rows['full'])
    # words = text.split()
    test_predictions.append(predicted_tags)

Making predictions: 100%|██████████| 30831/30831 [05:23<00:00, 95.27it/s]


In [105]:
len(test_predictions), len(test)

(30831, 30831)

In [106]:
test['pred_tags'] = test_predictions

In [108]:
def calculate_iou(predictions, true_labels):
    pred_set = set(predictions)
    true_set = set(true_labels)
    
    intersection = len(pred_set.intersection(true_set))
    union = len(pred_set.union(true_set))
    iou = intersection / union if union > 0 else 0
    return iou

In [109]:
test['IoU'] = test.apply(lambda row: calculate_iou(row['pred_tags'], row['bio_tags']), axis=1)
test.head()

,description,cooking_steps,ingredients,full,bio_tags,pred_tags,IoU
99449,тесто просто потрясать лёгкий идеально сочетат...,NaN,"[мука пшеничный, сметана, яйцо куриный, масло ...",тесто просто потрясать лёгкий идеально сочетат...,"[O, O, O, O, O, O, O]","[O, O, O, O, O, O, O]",1.0
37300,вкусный домашний воскресный пирожок,манка заливать молоко кефир минута добавлять я...,"[крупа манный, молоко, яйцо куриный, тыква, му...",вкусный домашний воскресный пирожок манка зали...,"[O, O, O, O, O, O, B-ING, O, O, O, O, B-ING, O...","[O, O, O, O, O, O, B-ING, B-ING, O, O, O, B-IN...",1.0
118788,давно хотеть попробовать сувид приготовление п...,нужный температура куриный филе градус мультив...,"[филе куриный, масло сливочный, розмарин, майо...",давно хотеть попробовать сувид приготовление п...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1.0
13695,увидеть супермаркет овощной смесь маседуан наз...,выглядеть набор продукт это овощной смесь масе...,"[краб, яйцо куриный, помидор, масло сливочный,...",увидеть супермаркет овощной смесь маседуан наз...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1.0
63911,киш рикоттой порционный рамекина микроволновка...,молоть овсянка мука соединять сухой ингредиент...,"[мука овсяный, мука пшеничный, крахмал картофе...",киш рикоттой порционный рамекина микроволновка...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1.0


In [111]:
test['IoU'].mean()

0.9503313764284865

Качество получилось очень высоким, в среднем IoU = 95%.

Сохраняем модель.

In [112]:
torch.save({'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),}, 
            'ingredient_model.pth')